In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv", low_memory = False)
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
faults_prepped = faults.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [5]:
exclude_coords = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.7  # Radius of Earth in km - roughly 3958.7 miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2 # this is the haversine formula that's accounting for the curvature of the earth. think calculus & a polar coordinate system.
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) # we're calculating the central angle between the two points
    return R * c # calculates the arc length (distance) we're multiplying the radius of earth and the central angle
threshold = 0.5  # Distance in km - this is roughly about 3.1 miles
distances = np.array([
    haversine(faults_prepped['Latitude'], faults_prepped['Longitude'], lat, lon)
    for lat, lon in exclude_coords
]) # I'm calculating the difference between the excluded coordinates and geo coordinate in my dataframe. the array is contains the distances as scalar quantities (floats).
# which rows are outside all vicinities
outside_vicinity = (distances > threshold).all(axis=0) # boolean array true if its outside vicinity
faults_prepped = faults_prepped[outside_vicinity]

In [6]:
diagnostics_pivot = diagnostics.pivot(index=['FaultId'], columns='Name', values='Value')

In [7]:
data = diagnostics_pivot.merge(faults_prepped, left_on = 'FaultId', right_on = 'RecordID', how = 'inner')

In [33]:
data.sample(5)

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,ExactMatch
864440,100.0,14.5725,False,66.48672,447221.3,179.6,0.0,40.02,211.10000,1409.875,9150.95,46.0,59360.384687,0.000000,32.0,True,62.6,1279,False,NaN,69.49648,NaN,100,0.87,992509,44325121,2018-03-31 08:37:32,High Voltage (Left Fuel Level Sensor),NaN,NaN,49,829,3,True,126,1723,105453385,36.280462,-86.803148,2018-03-31 08:36:54,False
700469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,796741,16803637,2017-05-30 10:01:05,Incorrect Data J1939 Network #1 Primary Vehicl...,BB41103* BB41104*,S221411139,11,639,2,False,127,1770,105442838,37.834907,-85.469074,2017-05-30 10:01:00,False
538310,0.0,14.5000,False,0.00000,304633.6,86.0,21.0,64.38,83.80625,649.500,6201.05,72.4,41117.455292,1.294447,77.0,True,73.4,17407,False,NaN,0.00000,NaN,0,0.58,610621,12195026,2016-10-15 04:21:04,Low (Severity Medium) Engine Coolant Level,PC4__1284P4C_2*,6U13D13,0,111,18,True,3,1745,105395973,33.203981,-84.064444,2016-10-15 04:21:41,False
897010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,1036002,55272948,2018-07-16 12:38:02,High Voltage (Left Fuel Level Sensor),NaN,NaN,49,829,3,False,126,1873,105410475,35.632037,-80.791018,2018-07-16 12:37:58,False
668729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,760446,15656838,2017-04-12 07:57:31,Abnormal Update Rate Tire Location,NaN,NaN,49,929,9,False,126,1641,105434215,39.987685,-81.952962,2017-04-12 07:57:27,False


In [9]:
columns_to_change = ['AcceleratorPedal', 
                     'BarometricPressure',
                     'CruiseControlSetSpeed',
                     'DistanceLtd',
                     'EngineCoolantTemperature',
                     'EngineLoad',
                     'EngineOilPressure',
                     'EngineOilTemperature',
                     'EngineRpm',
                     'EngineTimeLtd',
                     'FuelLevel',
                     'FuelLtd',
                     'FuelRate',
                     'FuelTemperature',
                     'IntakeManifoldTemperature',
                     'Speed',
                     'SwitchedBatteryVoltage',
                     'TurboBoostPressure',
                    ]   

In [10]:
for column in columns_to_change:
    data[column] = data[column].str.replace(',', '.').astype(np.float64)

In [23]:
data['EngineRpm'].describe()

count    531324.000000
mean        977.688730
std         404.159394
min           0.000000
25%         601.375000
50%        1078.625000
75%        1342.250000
max        2715.625000
Name: EngineRpm, dtype: float64

In [25]:
data.loc[data['spn'] == 1569]['EngineRpm'].describe()

count    4959.000000
mean     1076.714862
std       339.198449
min         0.000000
25%       650.750000
50%      1221.875000
75%      1357.312500
max      1881.625000
Name: EngineRpm, dtype: float64

In [27]:
data.loc[data['spn'] == 5246]['EngineRpm'].describe()

count     463.000000
mean      701.558045
std       424.685256
min         0.000000
25%       598.062500
50%       648.750000
75%      1002.500000
max      1626.500000
Name: EngineRpm, dtype: float64

In [29]:
data.loc[data['spn'] != 5246 & 1569]['EngineRpm'].describe()

count    530999.000000
mean        978.035437
std         403.960955
min           0.000000
25%         601.375000
50%        1079.250000
75%        1342.375000
max        2715.625000
Name: EngineRpm, dtype: float64

In [15]:
data['LocationTimeStamp'] = pd.to_datetime(data['LocationTimeStamp'])
data['EventTimeStamp'] = pd.to_datetime(data['EventTimeStamp'])

In [16]:
data = data.drop(['ecuModel','ecuMake', 'MCTNumber', 'ServiceDistance'], axis=1)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057049 entries, 0 to 1057048
Data columns (total 40 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AcceleratorPedal           480660 non-null   float64       
 1   BarometricPressure         530902 non-null   float64       
 2   CruiseControlActive        520765 non-null   object        
 3   CruiseControlSetSpeed      521823 non-null   float64       
 4   DistanceLtd                530760 non-null   float64       
 5   EngineCoolantTemperature   530890 non-null   float64       
 6   EngineLoad                 530421 non-null   float64       
 7   EngineOilPressure          531008 non-null   float64       
 8   EngineOilTemperature       529370 non-null   float64       
 9   EngineRpm                  531324 non-null   float64       
 10  EngineTimeLtd              527047 non-null   float64       
 11  FuelLevel                  455471 non

In [117]:
data['target_time'] = data.loc[data['spn'] == 5246]['EventTimeStamp'] - timedelta(hours=2)

In [119]:
data['target'] = data['target_time'].notna()

In [125]:
data.loc[data['target'] == True]

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,ExactMatch,target,target_time
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902,NaN,NaN,False,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23,False,True,2015-02-23 03:05:44
2589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58,False,True,2015-02-23 13:54:22
5141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN,5714,1070646,2015-02-25 13:53:08,NaN,unknown,unknown,0,5246,0,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31,False,True,2015-02-25 11:53:08
5236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,5810,1071907,2015-02-25 14:47:00,NaN,unknown,unknown,0,5246,0,False,1,1329,105400037,39.399629,-82.974814,2015-02-25 14:46:56,False,True,2015-02-25 12:47:00
5949,NaN,14.5000,False,64.62260,441699.6,185.0,10.0,20.30,198.16250,648.125,9087.95,57.2,69605.769379,0.634015,NaN,True,140.0,22527,NaN,NaN,0.0,3276.75,NaN,0.58,6535,1097942,2015-02-26 22:24:29,NaN,04993120*00021657*082113134117*07700053*I0*BBZ*,79466573,0,5246,0,True,1,1419,105355995,37.596805,-85.865555,2015-02-26 22:25:05,False,True,2015-02-26 20:24:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1239564,119961467,2020-02-06 08:03:09,NaN,04358814*06005963*051718174436*09401683*G1*BDR*,79897320,0,5246,0,False,1,1854,105385876,35.943472,-83.823240,2020-02-06 08:03:05,False,True,2020-02-06 06:03:09
1052278,0.0,13.9925,False,57.78752,477486.8,197.6,10.0,21.46,204.96880,599.500,9344.30,72.4,63966.356499,0.462303,NaN,True,120.2,22527,True,NaN,0.0,NaN,100,0.58,1241841,120905759,2020-02-13 13:32:39,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,0,5246,0,True,1,1872,105301976,35.707268,-81.397037,2020-02-13 13:33:15,False,True,2020-02-13 11:32:39
1052292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,1241858,120910417,2020-02-13 14:01:40,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,0,5246,0,False,1,1872,105301976,35.708101,-81.395648,2020-02-13 13:59:51,False,True,2020-02-13 12:01:40
1052528,0.0,14.7900,False,0.00000,334736.2,17.6,51.0,84.68,18.10625,651.500,11554.20,100.0,393.220099,2.892693,19.4,True,26.6,255,True,NaN,0.0,NaN,0,0.58,1242137,121038018,2020-02-14 11:21:54,NaN,NaN,NaN,49,5246,19,True,88,302,105418777,38.349490,-85.708425,2020-02-14 11:22:30,False,True,2020-02-14 09:21:54


In [137]:
data.loc[data['EventTimeStamp' == '2015-02-23 03:05:44']]

KeyError: False

In [ ]:
data_csv = pd.to_csv(